In [191]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import numpy as np

In [192]:
torch.manual_seed(42)

In [193]:
data=pd.read_csv('fmnist_small.csv')
data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [194]:
data.shape

(6000, 785)

In [195]:
x=data.drop(columns=['label']).values
y=data['label'].values

In [196]:
print(x.shape)
print(y.shape)


(6000, 784)
(6000,)


In [197]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [198]:
x_train=x_train/255.0
x_test=x_test/255.0

In [199]:
class CustomDataset(Dataset):
    def __init__(self,features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


In [200]:
train_dataset = CustomDataset(x_train, y_train)
test_dataset = CustomDataset(x_test, y_test)

In [201]:
len(train_dataset),
len(test_dataset)

1200

In [202]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [203]:
class MyNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.model=nn.Sequential(
            # 784->128->64->10
            nn.Linear(784, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64), 
            nn.ReLU(), 
            nn.Dropout(0.3),
            nn.Linear(64, 10)
        )
        
    def forward(self, x):
        return self.model(x)

In [204]:
learning_rate = 0.009
epochs=100

In [205]:
model=MyNN()


In [206]:
loss_fn=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate,weight_decay=1e-4)

In [207]:
for epoch in range(epochs):
    for batch_features,batch_labels in train_loader:
        y_pred=model(batch_features)
        loss=loss_fn(y_pred,batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")
        

Epoch 1/100, Loss: 0.6920
Epoch 2/100, Loss: 0.5166
Epoch 3/100, Loss: 0.5007
Epoch 4/100, Loss: 0.4732
Epoch 5/100, Loss: 0.8197
Epoch 6/100, Loss: 0.6527
Epoch 7/100, Loss: 0.6237
Epoch 8/100, Loss: 0.4230
Epoch 9/100, Loss: 0.7474
Epoch 10/100, Loss: 0.6837
Epoch 11/100, Loss: 0.3152
Epoch 12/100, Loss: 0.2077
Epoch 13/100, Loss: 0.5512
Epoch 14/100, Loss: 0.5242
Epoch 15/100, Loss: 0.6798
Epoch 16/100, Loss: 0.2646
Epoch 17/100, Loss: 0.9470
Epoch 18/100, Loss: 0.8144
Epoch 19/100, Loss: 0.5161
Epoch 20/100, Loss: 0.3245
Epoch 21/100, Loss: 0.1670
Epoch 22/100, Loss: 0.4867
Epoch 23/100, Loss: 0.3177
Epoch 24/100, Loss: 0.2455
Epoch 25/100, Loss: 0.3890
Epoch 26/100, Loss: 0.6306
Epoch 27/100, Loss: 0.5363
Epoch 28/100, Loss: 0.3321
Epoch 29/100, Loss: 0.3869
Epoch 30/100, Loss: 0.4075
Epoch 31/100, Loss: 0.7209
Epoch 32/100, Loss: 0.6005
Epoch 33/100, Loss: 0.5043
Epoch 34/100, Loss: 0.4368
Epoch 35/100, Loss: 0.5520
Epoch 36/100, Loss: 0.3590
Epoch 37/100, Loss: 0.4529
Epoch 38/1

In [208]:
model.eval()
correct=0
total=0
with torch.no_grad():
    for batch_features,batch_labels in test_loader:
        y_pred=model(batch_features)
        _, predicted=torch.max(y_pred.data,1)
        total+=batch_labels.shape[0]
        correct+=(predicted==batch_labels).sum().item()
accuracy=100*correct/total
print(f"Accuracy of the model on the test set: {accuracy:.2f}%")


Accuracy of the model on the test set: 80.67%
